<a href="https://colab.research.google.com/github/sandrakaku/ml0930/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [ ]:
import glob #找所有記事本並建立起來
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
# datas = { # 準備空的字典
#     "content":[],
#     "target":[]
# }
import pandas as pd

def read(path):
  with open(path, "r", encoding="utf-8") as f:
    content = f.read()
  return content


def get_data(t):
  dn = os.path.dirname(dataset)
  pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt") # 所有檔名
  pos = glob.glob(pattern)
  pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt") # 所有檔名
  neg = glob.glob(pattern)
  sentiments = [1] * len(pos) + [0] * len(neg)
  contents = map(read, pos + neg)
  df = pd.DataFrame({
      "contents":contents,
      "sentiment":sentiments
  })
  return df
train_df = get_data("train")
test_df = get_data("test")
test_df

In [ ]:
"-".join(map(str, [1, 2, 3, 4, 5])) # (與本主題無關)map對裡面的所有東西做一次相同的事

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# sequences: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [ ]:
# tok.word_index
tok.index_word

In [ ]:
# pd.DataFrame(train_df_seq)
# truncating: pre 截掉前面 ex: 12345 -> 45  / post 截掉後面 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences # 截長補短
x_train_pad = pad_sequences(x_train_seq, 512) # truncating(預設值就是"pre")
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

In [ ]:
# 以下開始enbadding

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding   # 詞意
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # param: 3001 * 128
  Embedding(3001, 128, mask_zero=True, input_length=512), # 128:output_dim, input_length=512(一篇文章有512個詞) 3000種512個 [每一個詞被畫成128個情緒]
  Flatten(),
  Dense(128, activation="relu"),
  Dropout(0.25),
  Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [ ]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

In [ ]:
model.evaluate(x_test_pad, y_test)